# Extraccion de los posters

Para extraer los posters, se realizo lo siguiente:
1. Del train, saco el id (que pertenece a MovieLens)
2. Como tengo el id de movielens, usando el links.csv, puedo hallar el imdbID y el tmdbID
3. Usando imdbID, desde el csv MovieGenre, puedo hallar el link del poster de manera directa
4. Si esto no funciona, uso el tmdbID para buscar el poster por la API 
5. Si no se encuentra, se omite.

## Librerías Necesarias

In [ ]:
import os
import requests
from tqdm import tqdm
import pandas as pd
from urllib.parse import urlparse

In [22]:
movies_train = pd.read_csv("../data/movies_train.csv")

In [23]:
movies_genre = pd.read_csv("../data/MovieGenre.csv", encoding='latin-1')

In [24]:
MovieLens = pd.read_csv("../data/movies.csv")
Links = pd.read_csv("../data/links.csv")

In [25]:
# Vincular movieId desde movies_train con links
merged = movies_train.merge(Links, on='movieId', how='left')
# Luego, vincular con movies_genre usando imdbId
merged = merged.merge(movies_genre[['imdbId', 'Poster']], on='imdbId', how='left')

In [26]:
merged.head()

,movieId,title,genres,(no genres listed),Action,Adventure,Animation,Children,Comedy,Crime,...,Musical,Mystery,Romance,Sci-Fi,Thriller,War,Western,imdbId,tmdbId,Poster
0,619,Ed (1996),Comedy,0,0,0,0,0,1,0,...,0,0,0,0,0,0,0,116165.0,32308.0,https://images-na.ssl-images-amazon.com/images...
1,33826,Saint Ralph (2004),Comedy|Drama,0,0,0,0,0,1,0,...,0,0,0,0,0,0,0,384488.0,25248.0,https://images-na.ssl-images-amazon.com/images...
2,1298,Pink Floyd: The Wall (1982),Drama|Musical,0,0,0,0,0,0,0,...,1,0,0,0,0,0,0,84503.0,12104.0,https://images-na.ssl-images-amazon.com/images...
3,140289,Men & Chicken (2015),Comedy|Drama,0,0,0,0,0,1,0,...,0,0,0,0,0,0,0,3877674.0,296313.0,https://images-na.ssl-images-amazon.com/images...
4,3064,Poison Ivy: New Seduction (1997),Drama|Thriller,0,0,0,0,0,0,0,...,0,0,0,0,1,0,0,119908.0,18222.0,https://images-na.ssl-images-amazon.com/images...


In [46]:
# Carpeta donde se guardarán los pósteres
output_dir = "Posterss"
os.makedirs(output_dir, exist_ok=True)

# Clave de API de TMDb
TMDB_API_KEY = "c9c68dece2621f327eb0f495c3d5bc42"


In [47]:
# DESCARGA DE POSTERES DESDE MOVIEGENRE
print("\n Descargando posters disponibles desde MovieGenre.csv...")

downloaded = 0
for _, row in tqdm(merged.iterrows(), total=len(merged)):
    title = row["title"]
    poster_url = row.get("Poster")

    if pd.notna(poster_url):
        file_path = os.path.join(output_dir, f"{title}.jpg")

        if not os.path.exists(file_path):
            try:
                response = requests.get(poster_url, timeout=5)
                if response.status_code == 200:
                    with open(file_path, "wb") as f:
                        f.write(response.content)
                    downloaded += 1
                else:
                    continue
            except:
                continue

print(f"Posters descargados desde MovieGenre: {downloaded}")



 Descargando posters disponibles desde MovieGenre.csv...


100%|██████████| 6943/6943 [29:29<00:00,  3.92it/s]   

Posters descargados desde MovieGenre: 798


In [49]:

# DESCARGA DE POSTERES FALTANTES USANDO TMDb
print("\n Buscando posters faltantes con TMDb API...")

img_base = "https://image.tmdb.org/t/p/w500"
base_url = "https://api.themoviedb.org/3/movie/{tmdb_id}"

downloaded_api = 0

for _, row in tqdm(merged.iterrows(), total=len(merged)):
    title = row["title"]
    file_path = os.path.join(output_dir, f"{title}.jpg")

    # Si ya existe el póster, se omite
    if os.path.exists(file_path):
        continue

    tmdb_id = row.get("tmdbId")
    if pd.notna(tmdb_id):
        url = base_url.format(tmdb_id=int(tmdb_id))
        params = {"api_key": TMDB_API_KEY}

        try:
            response = requests.get(url, params=params, timeout=8)
            if response.status_code == 200:
                data = response.json()
                if data.get("poster_path"):
                    poster_url = img_base + data["poster_path"]
                    img_data = requests.get(poster_url).content
                    with open(file_path, "wb") as f:
                        f.write(img_data)
                    downloaded_api += 1
        except:
            continue

print(f"Posters descargados desde TMDb: {downloaded_api}")



 Buscando posters faltantes con TMDb API...


100%|██████████| 6943/6943 [1:50:11<00:00,  1.05it/s]  

Posters descargados desde TMDb: 5694


In [53]:
poster_dir = "Posterss"

for f in os.listdir(poster_dir):
    path = os.path.join(poster_dir, f)
    
    # Saltar si es un directorio
    if os.path.isdir(path):
        print(f"⏭️ Saltando directorio: {f}")
        continue
    
    if not f.lower().endswith('.jpg'):
        os.remove(path)
        print(f"🗑️ Eliminado archivo que no es .jpg: {f}")
    elif os.path.getsize(path) < 500:
        os.remove(path)
        print(f"🗑️ Eliminado archivo .jpg corrupto: {f}")

🗑️ Eliminado archivo que no es .jpg: Oblivion 2
🗑️ Eliminado archivo que no es .jpg: Oh, Hello
🗑️ Eliminado archivo que no es .jpg: Old Men
🗑️ Eliminado archivo que no es .jpg: Once Upon a Time in China II (Wong Fei-hung Ji Yi
🗑️ Eliminado archivo que no es .jpg: Once Upon a Time in China III (Wong Fei-hung tsi sam
🗑️ Eliminado archivo que no es .jpg: Open Water 2
🗑️ Eliminado archivo que no es .jpg: OT
🗑️ Eliminado archivo que no es .jpg: Othello (Tragedy of Othello
🗑️ Eliminado archivo que no es .jpg: Pacific Rim
🗑️ Eliminado archivo que no es .jpg: Paradise Lost
🗑️ Eliminado archivo que no es .jpg: Paradise Lost 2
🗑️ Eliminado archivo que no es .jpg: Paradise Lost 3
🗑️ Eliminado archivo que no es .jpg: Parasyte
🗑️ Eliminado archivo que no es .jpg: Patlabor 2
🗑️ Eliminado archivo que no es .jpg: Patton Oswalt
🗑️ Eliminado archivo que no es .jpg: Paul Blart
🗑️ Eliminado archivo que no es .jpg: Pelé
🗑️ Eliminado archivo que no es .jpg: Percy Jackson
🗑️ Eliminado archivo que no es .jpg:

In [62]:
poster_dir = 'Posterss'
# Obtener los IDs de los posters descargados (nombres de archivo sin extensión)
posters_disponibles = {
    os.path.splitext(f)[0] for f in os.listdir(poster_dir) if f.endswith(".jpg")
}

# Filtrar el dataset: conservar solo los movieId con poster
movies_filtradas = movies_train[movies_train["title"].astype(str).isin(posters_disponibles)]

# Agregar columna 'poster' con la ruta al archivo
movies_filtradas["poster_path"] = movies_filtradas["title"].astype(str).apply(lambda x: f"{poster_dir}/{x}.jpg")

# Mostrar resumen
print(f"Películas originales: {len(movies_train)}")
print(f"Películas con poster:  {len(movies_filtradas)}")

# Guardar dataset final
movies_filtradas.to_csv("movies_train_with_posters.csv", index=False)

print("\n✅ Archivo 'movies_train_with_posters.csv' generado correctamente.")
print("Incluye solo películas con poster y una columna con la ruta local.")

Películas originales: 6819
Películas con poster:  5986

✅ Archivo 'movies_train_with_posters.csv' generado correctamente.
Incluye solo películas con poster y una columna con la ruta local.


C:\Users\USER\AppData\Local\Temp\ipykernel_23444\1664673478.py:11: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  movies_filtradas["poster_path"] = movies_filtradas["title"].astype(str).apply(lambda x: f"{poster_dir}/{x}.jpg")
